# 战争传播模型：
描述空间依赖与空间异质性的简易ABM模型
- 空间依赖性：指的是一个地区的周边发生内战，这个地区更容易发生内战
- 空间异质性：依据现有研究，我们把空间异质性定义为距离首都的远近。距离首都越近更容易发生战争，国家的上层发动战争、中央政府所在地往往在首都

## 导入包
导入mesa包、mesa-geo包

In [9]:
import mesa
import mesa_geo as mg
import mesa_geo.visualization as mgv

## 创建基于地区的agent

In [10]:
class regionAgent(mg.GeoAgent):
    #问题：是把阈值放进attribute还是把与agent相关的加进去
    #都要放进去
    #这里我想放地区的和平或战争状态、战争感染率、战争恢复率、首都的位置
    def __init__(self, unique_id, model, geometry, crs,agent_type ="no_war",war_infection_risk=0.1 ,war_recovery_rate=0.2):
        super().__init__(unique_id, model, geometry, crs)
        self.atype= agent_type
        self.war_infection_risk=war_infection_risk
        self.war_revovery_rate=war_recovery_rate
        self.in_war_step=0

    def __repr__(self):
        return "region" + str(self.unique_id)
    
    #如果距离未发生战争的agent固定距离内有发生战争的agent，也有可能直接发生战争（取决于阈值设定）
    #距离首都近的agent更容易发生战争（这里可以设定一个权重）、更贫穷更容易发生战争
    def war_spread(self):
        if self.atype == "no_war":
            neighbors= self.model.space.get_neighbors_within_distance(
                self,self.model.exposure_distance
            )
        #如果邻居里有一个以上发生战争，我该怎么表示？
        #遍历所有的邻居，然后再循环中赋予self.atype的值
            for neighnor in neighbors:
            #我这里想用这个函数distance(self, agent_a, agent_b)或者agent_a.geometry.distance(agent_b.geometry)，来表达agent距离首都的距离
            #现在的问题是：首都agent怎么找，根据id找吗？
                A_C_distance=self.model.space.distance(agent_a=self, agent_b=self.model.capitalAgent)
                if (neighnor.atype == "in_war" 
                    and A_C_distance*0.001+self.random.random() < self.model.threshold_war_infection
                    ):
                    self.atype = "in_war"
                    self.in_war_step=0
                    break
        
        #设置已经发生内战的区域的step
        elif self.atype=="in_war":
            if self.in_war_step<3:
                if self.random.random() < self.war_revovery_rate:
                    self.atype = "no_war"
                    self.in_war_step=self.in_war_step+1
            else:
                self.in_war_step=0
    
    def step(self):
        self.war_spread()
        self.model.counts[self.atype] += 1
   

## 基于mesa包创建模型，联系起agent、空间与时间序列

In [11]:
def get_in_war_count(model):
    return model.counts["in_war"]

def get_no_war_count(model):
    return model.counts["no_war"]

In [12]:
class war_model(mesa.Model):
    geojson_regions = "D:\编程的文档\python\mesa-geo库的学习\TorontoNeighbourhoods.geojson"
    unique_id = "HOODNUM"
    
    def __init__(self,int_war_rate=0.2,war_revovery_rate=0.6,threshold_war_infection=1.5,exposure_distance=1000):
        super().__init__()
        
        self.int_war_rate=int_war_rate
        self.war_recovery_rate=war_revovery_rate
        self.threshold_war_infection=threshold_war_infection
        self.exposure_distance=exposure_distance


        self.schedule=mesa.time.RandomActivation(self)
        self.space=mg.GeoSpace(warn_crs_conversion=False)
        #首都的id
        self.capital_id=105


        self.counts=None
        self.reset_counts()

        self.datacollector = mesa.DataCollector(
            {
                "no_war": get_no_war_count,
                "in_war": get_in_war_count
            }
        ) 


        #如何把参数加到地区agent里
        ac=mg.AgentCreator(
            regionAgent,
            model=self)  
        region_agents = ac.from_file(self.geojson_regions,unique_id=self.unique_id)
        
        for j in range(len(region_agents)):
            if self.random.random() < self.int_war_rate:
             #if self.random.random() < 0.2:
                region_agents[j].atype="in_war"

        self.space.add_agents(region_agents)

        #找到capital agent首都
        self.capitalAgent = next((agent for agent in region_agents if agent.unique_id == self.capital_id), None)
        #capital_agent=[region_agents for agent in region_agents if agent.unique_id == self.capital_id]
        #self.capitalAgent= capital_agent[0]    

        for agent in region_agents:
            self.schedule.add(agent)


    def reset_counts(self):
        self.counts = {
            "in_war": 0,
            "no_war":0
            }
        
    def step(self):
        self.reset_counts()
        self.schedule.step()
        self.datacollector.collect(self) 

    



        




## 运行战争spread模型

In [13]:
model = war_model()
for i in range(10): 
    model.step()

## 战争spread模型可视化

In [14]:
model.datacollector.get_model_vars_dataframe()

,no_war,in_war
0,1,4
1,1,4
2,3,2
3,3,2
4,2,3
5,2,3
6,1,4
7,1,4
8,3,2
9,2,3


In [15]:
def war_model_draw(agent):
    """
    Portrayal Method for canvas
    """

    portrayal = {}       
    if isinstance(agent, regionAgent):
        if agent.atype == "in_war":
            portrayal["color"] = "Red"
        else: 
            portrayal["color"] = "green"
    
    return portrayal


model_params = {
    "int_war_rate": {
        "type": "SliderFloat",
        "value": 0.2,
        "label": "Population Size",
        "min": 0,
        "max": 0.6, 
        "step": 0.05,
    },
    "threshold_war_infection": {
        "type": "SliderFloat",
        "value": 1.5,
        "label": "Population Size",
        "min": 1.2,
        "max": 2, 
        "step": 0.1,
    },
    "exposure_distance": {
        "type": "SliderInt",
        "value": 1000,
        "label": "Population Size",
        "min": 500,
        "max": 2000, 
        "step": 100,
    },
    "war_revovery_rate": {
      "type": "SliderFloat",
      "value": 0.6,
        "label": "Maximum Number of Steps to Recover",
        "min": 0.1,
        "max": 1, 
        "step": 0.1,   
     }}

In [16]:
page = mgv.GeoJupyterViz(
    war_model,
    model_params,
    measures= [ ["in_war", "no_war"]],
    name="war spread",   #界面名
    agent_portrayal=war_model_draw,  
    zoom=12,
    scroll_wheel_zoom=False
)
# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text

## reference

参考文献：
陈冲，胡竞天；空间依赖与武装冲突预测